In [21]:
import requests 
import datetime
import pandas as pd
import altair as alt
import json


from dotenv import load_dotenv
import os

load_dotenv()

# api_key=os.getenv("WEATHERAPI_APIKEY")

True

In [22]:
# Historical data does not work with the free key
# response = requests.get(f"https://api.weatherapi.com/v1/current.json?key={api_key}&q=Berlin&dt=2015-01-01")
# berlin = response.json()

In [23]:
# I thought about doing an API-Call, but downloading the csv was just so much easier. 

# response = requests.get('https://archive-api.open-meteo.com/v1/archive?latitude=52.52&longitude=13.405&start_date=2015-06-16&end_date=2025-06-15&daily=temperature_2m_mean,rain_sum,snowfall_sum,sunshine_duration')

# berlin = response.json()

# response = requests.get('https://archive-api.open-meteo.com/v1/archive?latitude=40.6782&longitude=73.9442&start_date=2015-06-16&end_date=2025-06-15&daily=temperature_2m_mean,rain_sum,snowfall_sum,sunshine_duration')

# new_york = response.json()

Data source: https://open-meteo.com/

This describtion works for both dataframes - one will have berlin added, the other one will have brooklyn added

day: date of the measures
temperature: mean temperature on that day in Celcius
rain: sum of rain on that day in millimeter
snowfall: sum of snowfall on that day in Centimeter
sunshine: duration of sunshine in that day in hours (converted von seconds to hours)

In [24]:
df = pd.read_csv('raw/weather_brooklyn.csv', header=2)

In [25]:
# rename columns

(
    df
    .rename(
        columns={
            'time': 'day', 
            'temperature_2m_mean (°C)' : 'temperature', 
            'rain_sum (mm)' : 'rain', 
            'snowfall_sum (cm)': 'snowfall', 
            'sunshine_duration (s)':'sunshine',
            'weather_code (wmo code)' : 'weather_code',
            'relative_humidity_2m_mean (%)' : 'humidity',
            'apparent_temperature_mean (°C)' : 'feels_like',
            'daylight_duration (s)' : 'daylight_duration'
        }, 
        inplace=True
    )
)

In [26]:
df

,day,weather_code,temperature,feels_like,daylight_duration,sunshine,snowfall,rain,humidity
0,2015-06-15,63,22.1,24.7,54245.02,17874.43,0.0,21.7,89
1,2015-06-16,63,21.6,24.0,54265.15,19290.01,0.0,8.9,88
2,2015-06-17,3,20.9,21.5,54281.74,32400.00,0.0,0.0,72
3,2015-06-18,3,18.6,18.7,54294.77,14856.77,0.0,0.0,77
4,2015-06-19,51,22.8,24.6,54304.23,41578.02,0.0,0.3,76
...,...,...,...,...,...,...,...,...,...
3649,2025-06-11,3,23.7,24.3,54153.83,50199.27,0.0,0.0,57
3650,2025-06-12,51,27.4,27.4,54187.14,46895.38,0.0,0.1,46
3651,2025-06-13,3,23.6,22.5,54216.68,46193.10,0.0,0.0,43
3652,2025-06-14,53,18.1,17.5,54242.38,0.00,0.0,5.1,72


In [27]:
# calculate sunshine from seconds to hours

df['sunshine_hours'] = df['sunshine'] / 3600
df = df.drop('sunshine', axis=1)

In [28]:
# calculate daylight from seconds to hours

df['daylight_hours'] = df['daylight_duration'] / 3600
df = df.drop('daylight_duration', axis=1)

In [29]:
# add month to data
df['day'] = pd.to_datetime(df['day'])

In [30]:
df['month_year'] = df['day'].dt.to_period('M')
df['year'] = df['day'].dt.to_period('Y')

In [31]:
# add season according to month
# Dez, Jan, Feb == Winter
# Mar, Apr, Mai == Spring
# Jun, Jul, Aug == Summer
# Sep, Oct, Nov == Fall

df['month_num'] = df['day'].dt.month

def get_season(month):
    if month in [12, 1, 2]:
        return 'winter'
    elif month in [3, 4, 5]:
        return 'spring'
    elif month in [6, 7, 8]:
        return 'summer'
    elif month in [9, 10, 11]:
        return 'autumn'

df['season'] = df['month_num'].apply(get_season)

In [32]:
# match weathercodes with description




In [33]:
df.dtypes

day               datetime64[ns]
weather_code               int64
temperature              float64
feels_like               float64
snowfall                 float64
rain                     float64
humidity                   int64
sunshine_hours           float64
daylight_hours           float64
month_year             period[M]
year               period[Y-DEC]
month_num                  int32
season                    object
dtype: object

In [34]:
df['day_of_week'] = df['day'].dt.day_name()

In [35]:
df

,day,weather_code,temperature,feels_like,snowfall,rain,humidity,sunshine_hours,daylight_hours,month_year,year,month_num,season,day_of_week
0,2015-06-15,63,22.1,24.7,0.0,21.7,89,4.965119,15.068061,2015-06,2015,6,summer,Monday
1,2015-06-16,63,21.6,24.0,0.0,8.9,88,5.358336,15.073653,2015-06,2015,6,summer,Tuesday
2,2015-06-17,3,20.9,21.5,0.0,0.0,72,9.000000,15.078261,2015-06,2015,6,summer,Wednesday
3,2015-06-18,3,18.6,18.7,0.0,0.0,77,4.126881,15.081881,2015-06,2015,6,summer,Thursday
4,2015-06-19,51,22.8,24.6,0.0,0.3,76,11.549450,15.084508,2015-06,2015,6,summer,Friday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3649,2025-06-11,3,23.7,24.3,0.0,0.0,57,13.944242,15.042731,2025-06,2025,6,summer,Wednesday
3650,2025-06-12,51,27.4,27.4,0.0,0.1,46,13.026494,15.051983,2025-06,2025,6,summer,Thursday
3651,2025-06-13,3,23.6,22.5,0.0,0.0,43,12.831417,15.060189,2025-06,2025,6,summer,Friday
3652,2025-06-14,53,18.1,17.5,0.0,5.1,72,0.000000,15.067328,2025-06,2025,6,summer,Saturday


In [36]:
with open("raw/descriptions.json") as f:
    weather_dict = json.load(f)

In [37]:
weather_dict

{'0': {'day': {'description': 'Sunny',
   'image': 'http://openweathermap.org/img/wn/01d@2x.png'},
  'night': {'description': 'Clear',
   'image': 'http://openweathermap.org/img/wn/01n@2x.png'}},
 '1': {'day': {'description': 'Mainly Sunny',
   'image': 'http://openweathermap.org/img/wn/01d@2x.png'},
  'night': {'description': 'Mainly Clear',
   'image': 'http://openweathermap.org/img/wn/01n@2x.png'}},
 '2': {'day': {'description': 'Partly Cloudy',
   'image': 'http://openweathermap.org/img/wn/02d@2x.png'},
  'night': {'description': 'Partly Cloudy',
   'image': 'http://openweathermap.org/img/wn/02n@2x.png'}},
 '3': {'day': {'description': 'Cloudy',
   'image': 'http://openweathermap.org/img/wn/03d@2x.png'},
  'night': {'description': 'Cloudy',
   'image': 'http://openweathermap.org/img/wn/03n@2x.png'}},
 '45': {'day': {'description': 'Foggy',
   'image': 'http://openweathermap.org/img/wn/50d@2x.png'},
  'night': {'description': 'Foggy',
   'image': 'http://openweathermap.org/img/wn/50

In [38]:
df['weather_code'] = df['weather_code'].astype(str)

In [39]:
df['weather_description'] = df['weather_code'].map( lambda x: weather_dict.get(x, {}).get('day', {}).get('description', 'Unknown'))
df['weather_description']

0                Rain
1                Rain
2              Cloudy
3              Cloudy
4       Light Drizzle
            ...      
3649           Cloudy
3650    Light Drizzle
3651           Cloudy
3652          Drizzle
3653    Heavy Drizzle
Name: weather_description, Length: 3654, dtype: object

In [40]:
# save cleaned file with calculations

df.to_csv('processed/brooklyn_cleaned.csv', index=False)